In [8]:
%load_ext autoreload

In [9]:
%autoreload 2

In [10]:
from TSFEA import *

In [11]:
tsfea = TSFEA()

# Get Earnings Number to Homogenize Dates across Tickers

In [12]:
homogenized_earnings = tsfea.homogenize_earnings_dates('ActualEarningsData.csv')
homogenized_earnings.head()

CUSIP  OFTIC         CNAME     PENDS MEASURE PDICITY  \
TICKER ANNDATS                                                             
0001   20200325  26878510  EPEGQ  EP ENGR CORP  20191231     EPS     QTR   
000V   20220804  28249U10   EIGR         EIGER  20220630     EPS     QTR   
       20220505  28249U10   EIGR         EIGER  20220331     EPS     QTR   
       20220310  28249U10   EIGR         EIGER  20211231     EPS     QTR   
       20211104  28249U10   EIGR         EIGER  20210930     EPS     QTR   

                 VALUE  ERNUM  
TICKER ANNDATS                 
0001   20200325    NaN     15  
000V   20220804  -0.51     15  
       20220505  -0.64     14  
       20220310  -0.64     13  
       20211104  -0.65     12

# Reconcile Estimates Dates

In [13]:
reconciled_estimates = tsfea.reconcile_estimates_with_earnings(
    'InitialEstimatesData.csv',
    homogenized_earnings
)

In [15]:
reconciled_estimates.head()

OFTIC     CUSIP        CNAME  STATPERS MEASURE FISCALP ESTFLAG  \
TICKER ERNUM                                                                  
0045   12       PE  70187710  PARSLEY EGY  20180118     EPS     QTR       P   
       12       PE  70187710  PARSLEY EGY  20180215     EPS     QTR       P   
       12       PE  70187710  PARSLEY EGY  20180315     EPS     QTR       P   
       12       PE  70187710  PARSLEY EGY  20180419     EPS     QTR       P   
       12       PE  70187710  PARSLEY EGY  20180118     EPS     QTR       P   

              NUMEST  NUMUP  NUMDOWN  MEDEST  MEANEST  STDEV  HIGHEST  LOWEST  \
TICKER ERNUM                                                                    
0045   12         30     13        1    0.18     0.19   0.04     0.26    0.09   
       12         31      6       18    0.17     0.17   0.03     0.25    0.09   
       12         29     13       10    0.22     0.22   0.06     0.33    0.13   
       12         30      9        6    0.24     0.23   0.04     0.31    0.13   
       12         27     10        0    0.21     0.23   0.07     0.38    0.12   

              ACTUAL  ANNDATS_ACT  
TICKER ERNUM                       
0045   12       0.30   20180221.0  
       12       0.30   20180221.0  
       12       0.31   20180503.0  
       12       0.31   20180503.0  
       12       0.31   20180503.0